In [37]:
import sys
import importlib
import nltk

nltk.download('punkt') # Example download

# Add the shared folder to path
sys.path.append('../shared')

# Force reload the module to get the latest changes
if 'NLPMethods' in sys.modules:
    importlib.reload(sys.modules['NLPMethods'])

from NLPMethods import NLPMethods

# Create an instance of the NLPMethods class
nlp_methods = NLPMethods()

# Call the hello_from_class method
nlp_methods.hello_from_class()

clean_corpus = nlp_methods.remove_gutenberg_header("https://www.gutenberg.org/cache/epub/2500/pg2500.txt")

# Fetch the corpus and store the results
corpus_data = nlp_methods.fetch_corpus(clean_corpus)

# print(corpus_data['quote_sentences'])
chapters = nlp_methods.get_chapters(clean_corpus)
chapters_data = nlp_methods.get_chapter_data(chapters, clean_corpus)
chapters_df = nlp_methods.chapters_to_dataframe(chapters_data)
random_sample = nlp_methods.get_random_sample_chapter_data(chapters, clean_corpus)
systematic_sample = nlp_methods.get_systematic_sample_chapter_data(chapters, clean_corpus)
print(nlp_methods.compare_sample_lengths(random_sample, systematic_sample))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marobinette/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


hello from class
Number of sentences: 1857
Number of quotes found: 372
Quote sentences: 95
Non-quote sentences: 1762
Quote tokens: 0
Non-quote tokens: 39799
{'random_sample': {'count': 8, 'avg_tokens': 3227.75, 'avg_words': 3179.5, 'avg_chars': 17862.0}, 'systematic_sample': {'count': 8, 'avg_tokens': 3227.75, 'avg_words': 3179.5, 'avg_chars': 17862.0}, 'differences': {'token_diff': 0.0, 'word_diff': 0.0, 'char_diff': 0.0}}
